In [ ]:
import os

import pandas as pd
import json

In [20]:
with open('data/anthropopathic_data/complete_twitter_users.json') as f:
    twitter = json.load(f)

In [21]:
len(twitter)

167

In [22]:
c = 0
for t in twitter:
    c += len(t['likes'])
    c += len(t['timeline'])
    c += len(t['mentions'])
    # for x in t['likes']:
    #     x['type'] = 'likes'
    # for x in t['timeline']:
    #     x['type'] = 'timeline'
    # for x in t['mentions']:
    #     x['type'] = 'mentions'
print(c)

14601


In [ ]:
t = twitter[28]
ut =  t['mentions'] + t['likes'] + t['timeline']
sut = sorted(ut, key=lambda x: x['created_at'])
[s['created_at'] + ' ' + s['type'] for s in sut]

In [ ]:
texts = []
created_ats = []
ids = []
authors = []
from_users = []
from_user_ids = []
for t in twitter:
    for x in t['likes']:
        x['type'] = 'likes'
    for x in t['timeline']:
        x['type'] = 'timeline'
    for x in t['mentions']:
        x['type'] = 'mentions'
    for l in t['likes'] + t['timeline'] + t['mentions']:
        texts.append(l['text'])
        created_ats.append(l['created_at'])
        ids.append(l['id'])
        authors.append(l['author_id'])
        from_users.append(t['username'])
        from_user_ids.append(t["id"])
env_posts = pd.DataFrame({"id": ids, "author": authors, "text": texts, "created_at": created_ats, "from_user": from_users, "from_user_id": from_user_ids})

In [ ]:
env_posts.to_csv('toy_dir/twitter/env_posts.csv', index=False)

In [ ]:
user_cols = ['profile_image_url', 'username', 'description', 'id', 'protected', 'created_at', 'name', 'verified', 'follower_count', 'member_count', 'owner_id']
users = pd.DataFrame(columns=user_cols)
for item in twitter:
    data_to_add = {key: item[key] for key in user_cols}
    users = users.append(data_to_add, ignore_index=True)


In [ ]:
users.to_csv('toy_dir/twitter/env_users.csv', index=False)

### Reddit

In [23]:
with open('data/anthropopathic_data/reddit_user_popular_706.json', 'r', encoding='utf-8') as f:
    reddit = json.load(f)

In [24]:
len(reddit)

706

In [ ]:
import utils

In [ ]:
rd['actions']

In [ ]:
for rd in reddit:
    character_path = os.path.join('toy_dir/reddit/characters', rd['username'])
    utils.create_directory_if_not_exists(character_path)
    profile = {
        "interested_topics": rd['topics']
    }
    
    ca = sorted(rd['actions'], key=lambda x: x['publish_time'])
    save_actions = []
    for a in ca:
        name = a['type']
        act_time = a['publish_time']
        if name == "comment":
            action_args = {"content": a['content'], "root":  (a['root_parent'] or "") + (a['parent'] or "")}
        elif name == "submission":
            action_args = {"content": a['content']}
        else:
            raise Exception()
        save_actions.append({"name": name, "act_time": act_time, "action_args": action_args})
    with open(os.path.join(character_path, 'actions.json'), 'w', encoding='utf-8') as f:
        json.dump(save_actions, f, ensure_ascii=False)
    with open(os.path.join(character_path, 'profile.json'), 'w', encoding='utf-8') as f:
        json.dump(profile, f, ensure_ascii=False)

In [ ]:
a

In [ ]:
len(reddit)

## 知乎

In [ ]:
with open('data/anthropopathic_data/zhihu.json', 'r', encoding='utf-8') as f:
    zhihu = json.load(f)

In [19]:
c = 0
for zh in zhihu:
    c += len(zh['actions'])
print(c)

43367


In [ ]:
zh = zhihu[0]
character_path = os.path.join('toy_dir/zhihu/characters', zh['urlToken'])
os.mkdir(character_path)

In [ ]:
zh['location']

In [ ]:
import utils

In [ ]:
users = []
for zh in zhihu:
    user = zh.copy()
    user["interested_topics"] = [list(topic.keys())[0] for topic in zh['topics']]
    del user["actions"]
    users.append(user)
users = pd.DataFrame(users)
users = users.drop_duplicates(subset=['name'])
users.to_csv('toy_dir/zhihu/env_users.csv', index=False)

In [16]:
users[0]['action']

KeyError: 0

In [ ]:
posts = []
for zh in zhihu:
    ca = zh['actions']
    for a in ca:
        post = dict()
        post['author'] = ""
        post['text'] = (a['action_title'] or "") + (a['action_content'] or "")
        post['created_at'] = ""
        post['from_user'] = zh['name']
        post['from_user_id'] = zh['urlToken']
        posts.append(post)
        
posts = pd.DataFrame(posts)
posts["id"] = posts.index
posts = posts.drop_duplicates(subset=['text'])
posts.to_csv('toy_dir/zhihu/env_posts.csv', index=False)    

In [ ]:
posts.index

In [ ]:
for zh in zhihu:
    character_path = os.path.join('toy_dir/zhihu/characters', zh['name'])
    utils.create_directory_if_not_exists(character_path)
    profile = {
        'name': zh['name'],
        'description': (zh['headline'] or "") + ';' + (zh['description'] or ""),
        'gender': 'man' if zh['gender'] == 1 else 'woman' if zh['gender'] == 0 else 'unknown',
        'occupation': (zh['business'] or "") + '&' + (zh['employment'] or ""),
        'education': (zh['education'] or ""),
        'location': (zh['ip_location'] or "") + '&' + (zh['location'] or ""),
        "interested_topics": [list(topic.keys())[0] for topic in zh['topics']]
    }
    
    action_map = {"赞同了回答": "like", "回答了问题": "comment", "发布了想法": "post", "提出了问题": "post", "发表了文章": "post", "关注了问题": "follow_topic", "关注了收藏夹": "follow_topic", "赞同了文章": "like", "收藏了回答": "follow_topic", "添加了问题": "post", "收藏了文章": "follow_topic", "认可了回答": "like"}
    ca = sorted(zh['actions'], key=lambda x: x['action_time'])
    save_actions = []
    for a in ca:
        name = action_map[a['action_type']]
        act_time = a['action_time']
        a['action_title'] = '' if a['action_title'] is None else a['action_title']
        a['action_content'] = '' if a['action_content'] is None else a['action_content']
        a['action_content'] = a['action_content'][0]['content'] if isinstance(a['action_content'], list) else a['action_content']
        if name == "like":
            action_args = {"root": a['action_title'] + a['action_content']}
        elif name == "comment":
            action_args = {"root": a['action_title'] + a['action_content']}
        elif name == "post":
            action_args = {"content": a['action_title'] + a['action_content']}
        elif name == "follow_topic":
            action_args = {"keyword": a['action_title'] + a['action_content']}
        else:
            raise Exception("No use action name")
        save_actions.append({"name": name, "act_time": act_time, "action_args": action_args})
    with open(os.path.join(character_path, 'actions.json'), 'w', encoding='utf-8') as f:
        json.dump(save_actions, f, ensure_ascii=False)
    with open(os.path.join(character_path, 'profile.json'), 'w', encoding='utf-8') as f:
        json.dump(profile, f, ensure_ascii=False)

In [ ]:
a

In [ ]:
save_actions

In [ ]:
from actions import *

In [ ]:
for action in ca:
    if action['action_type'] == '赞同了回答':
        ao = LikeAction()
        ao.action_args = {"content": action['action_title'] + '\n' + action['action_content']}
        break

In [ ]:
ao.action_time

In [ ]:
def zhihu_action_build(actions: dict):
    f

In [ ]:
actions